<h1>Imports</h1>

In [2]:
# Importing necessary packages

import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

<h1>Supplementary Functions</h1>

In [3]:
# Calculating angle between joints

def calculate_angle(point1, point2, point3):
    point1 = np.array(point1)
    point3 = np.array(point3)
    point2 = np.array(point2)
    
    radians = np.arctan2(point3[1] - point2[1], point3[0] - point2[0]) - np.arctan2(point1[1] - point2[1], point1[0] - point2[0])
    angle = np.abs((radians * 180.0)/np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle

<h2>Initial Testing to differentiate state angles</h2>

In [7]:
cap = cv2.VideoCapture("sample_videos/abdullah_footage_1.mp4")

elbow_angles = []
shoulder_angles = []
hip_angles = []
knee_angles = []

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        results = pose.process(image)
        image.flags.writeable = True
        
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            # For Right side Points Visibility
            
            right_elbow_vis = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].visibility
            right_shoulder_vis = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].visibility
            right_hip_vis = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].visibility
            right_knee_vis = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].visibility
            right_wrist_vis = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].visibility
            right_ankle_vis = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].visibility
            
            
            # For Left side Points Visibility
            
            left_elbow_vis = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].visibility
            left_shoulder_vis = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility
            left_hip_vis = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].visibility
            left_knee_vis = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].visibility
            left_wrist_vis = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].visibility
            left_ankle_vis = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].visibility
            
            left_vis_sum = (left_hip_vis + left_knee_vis + left_ankle_vis + left_elbow_vis + left_wrist_vis + left_shoulder_vis) 
            
            # Right Keypoints
            
            right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y
            right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y
            right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y
            right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y
            right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y
            right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y
            
            right_vis_sum = (right_hip_vis + right_knee_vis + right_ankle_vis + right_elbow_vis + right_wrist_vis + right_shoulder_vis)
            
            # Left Keypoints
            
            left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y
            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y
            left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y
            left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
            left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y
            left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y
            
                
            
            # Elbow Angle Calculation
            
            if right_vis_sum >= left_vis_sum:
                elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
                elbow_angles.append(elbow_angle.round(1))
            else:
                elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
                elbow_angles.append(elbow_angle.round(1))
                
            
                
                
            # Shoulder Angle Calculation
            
            if right_vis_sum >= left_vis_sum:
                shoulder_angle = calculate_angle(right_hip, right_shoulder, right_elbow)
                shoulder_angles.append(shoulder_angle.round(1))
            else:
                shoulder_angle = calculate_angle(left_hip, left_shoulder, left_elbow)
                shoulder_angles.append(shoulder_angle.round(1))
        
                
                
            # Hip Angle Calculation
            
            if right_vis_sum >= left_vis_sum:
                hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)
                hip_angles.append(hip_angle.round(1))
            else:
                hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
                hip_angles.append(hip_angle.round(1))
                
            
            # Knee Angle Calculation
            
            
            if right_vis_sum >= left_vis_sum:
                knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
                knee_angles.append(knee_angle.round(1))
            else:
                knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
                knee_angles.append(knee_angle.round(1))
                        
            
            
        except:
            pass 
        
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        cv2.imshow('Raw Webcam Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
    cap.release()
    cv2.destroyAllWindows()
    
        
# print min and max angles for each joint

print("Elbow Angle: ", min(elbow_angles), max(elbow_angles))
print("Shoulder Angle: ", min(shoulder_angles), max(shoulder_angles))
print("Hip Angle: ", min(hip_angles), max(hip_angles))
print("Knee Angle: ", min(knee_angles), max(knee_angles))

Elbow Angle:  61.3 173.4
Shoulder Angle:  0.2 61.4
Hip Angle:  149.2 169.4
Knee Angle:  171.3 180.0


In [8]:
state1_angle_threshold = [150.0, 180.0]
state2_angle_threshold = [100.0, 130.0]
state3_angle_threshold = [50.0, 80.0]

<p>Using elbow angles will be better to transition through states.</p>
<p>This is because hips and knees barely move whereas elbow angles can vary throughout the movement.</p>

<h2>Getting Angles from Trainer Footage</h2>

In [3]:
trainer_elbow_angle_state1 = []
trainer_shoulder_angle_state1 = []
trainer_hip_angle_state1 = []
trainer_knee_angle_state1 = []

trainer_elbow_angle_state2 = []
trainer_shoulder_angle_state2 = []
trainer_hip_angle_state2 = []
trainer_knee_angle_state2 = []

trainer_elbow_angle_state3 = []
trainer_shoulder_angle_state3 = []
trainer_hip_angle_state3 = []
trainer_knee_angle_state3 = []

In [ ]:
cap = cv2.VideoCapture(0)

state = 0

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        results = pose.process(image)
        image.flags.writeable = True
        
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            # For Right side Points Visibility
            
            right_elbow_vis = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].visibility
            right_shoulder_vis = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].visibility
            right_hip_vis = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].visibility
            right_knee_vis = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].visibility
            right_wrist_vis = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].visibility
            right_ankle_vis = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].visibility
            
            right_vis_sum = (right_hip_vis + right_knee_vis + right_ankle_vis + right_elbow_vis + right_wrist_vis + right_shoulder_vis)
            
            # For Left side Points Visibility
            
            left_elbow_vis = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].visibility
            left_shoulder_vis = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility
            left_hip_vis = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].visibility
            left_knee_vis = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].visibility
            left_wrist_vis = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].visibility
            left_ankle_vis = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].visibility
            
            left_vis_sum = (left_hip_vis + left_knee_vis + left_ankle_vis + left_elbow_vis + left_wrist_vis + left_shoulder_vis) 
            
            # Right Keypoints
            
            right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y
            right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y
            right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y
            right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y
            right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y
            right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y
            
            
            # Left Keypoints
            
            left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y
            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y
            left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y
            left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
            left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y
            left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y
            
                
            
            # Elbow Angle Calculation and state selection
            
            elbow_angle = 0
            
            if right_vis_sum >= left_vis_sum:
                elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            else:
                elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)  
            
            
            if elbow_angle >= state1_angle_threshold[0] and elbow_angle <= state1_angle_threshold[1]:
                state = 1
            elif elbow_angle >= state2_angle_threshold[0] and elbow_angle <= state2_angle_threshold[1]:
                state = 2
            elif elbow_angle >= state3_angle_threshold[0] and elbow_angle <= state3_angle_threshold[1]:
                state = 3
            else:
                state = 0
                    
            if state == 1:
                trainer_elbow_angle_state1.append(elbow_angle.round(1))
            elif state == 2:
                trainer_elbow_angle_state2.append(elbow_angle.round(1))
            elif state == 3:
                trainer_elbow_angle_state3.append(elbow_angle.round(1))
            
                
            # Shoulder Angle Calculation
            
            shoulder_angle = 0
            
            if right_vis_sum >= left_vis_sum:
                shoulder_angle = calculate_angle(right_hip, right_shoulder, right_elbow)
            else:
                shoulder_angle = calculate_angle(left_hip, left_shoulder, left_elbow)
                
            if state == 1:
                trainer_shoulder_angle_state1.append(shoulder_angle.round(1))
            elif state == 2:
                trainer_shoulder_angle_state2.append(shoulder_angle.round(1))
            elif state == 3:
                trainer_shoulder_angle_state3.append(shoulder_angle.round(1)) 
            
                
            # Hip Angle Calculation
            
            hip_angle = 0
            
            if right_vis_sum >= left_vis_sum:
                hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)
            else:
                hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
                
            if state == 1:
                trainer_hip_angle_state1.append(hip_angle.round(1))
            elif state == 2:
                trainer_hip_angle_state2.append(hip_angle.round(1))
            elif state == 3:
                trainer_hip_angle_state3.append(hip_angle.round(1))
            
            # Knee Angle Calculation
            
            knee_angle = 0
            
            if right_vis_sum >= left_vis_sum:
                knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
            else:
                knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
                        
            if state == 1:
                trainer_knee_angle_state1.append(knee_angle.round(1))
            elif state == 2:
                trainer_knee_angle_state2.append(knee_angle.round(1))
            elif state == 3:
                trainer_knee_angle_state3.append(knee_angle.round(1))
            
            
        except:
            pass 
        
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        cv2.imshow('Raw Webcam Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
# print all trainer angle arrays

print("State 1:-\n")
print("Trainer Elbow Angle State 1: ", trainer_elbow_angle_state1)
print("Trainer Shoulder Angle State 1: ", trainer_shoulder_angle_state1)
print("Trainer Hip Angle State 1: ", trainer_hip_angle_state1)
print("Trainer Knee Angle State 1: ", trainer_knee_angle_state1)

print("\nState 2:-\n")
print("Trainer Elbow Angle State 2: ", trainer_elbow_angle_state2)
print("Trainer Shoulder Angle State 2: ", trainer_shoulder_angle_state2)
print("Trainer Hip Angle State 2: ", trainer_hip_angle_state2)
print("Trainer Knee Angle State 2: ", trainer_knee_angle_state2)

print("\nState 3:-\n")
print("Trainer Elbow Angle State 3: ", trainer_elbow_angle_state3)
print("Trainer Shoulder Angle State 3: ", trainer_shoulder_angle_state3)
print("Trainer Hip Angle State 3: ", trainer_hip_angle_state3)
print("Trainer Knee Angle State 3: ", trainer_knee_angle_state3)


In [ ]:
# print min and max angles for each joint

print("Min and max of angles:-\n")
print("\nState 1:-\n")
print("Elbow Angle: ", min(trainer_elbow_angle_state1), max(trainer_elbow_angle_state1))
print("Shoulder Angle: ", min(trainer_shoulder_angle_state1), max(trainer_shoulder_angle_state1))
print("Hip Angle: ", min(trainer_hip_angle_state1), max(trainer_hip_angle_state1))
print("Knee Angle: ", min(trainer_knee_angle_state1), max(trainer_knee_angle_state1))

print("\nState 2:-\n")
print("Elbow Angle: ", min(trainer_elbow_angle_state2), max(trainer_elbow_angle_state2))
print("Shoulder Angle: ", min(trainer_shoulder_angle_state2), max(trainer_shoulder_angle_state2))
print("Hip Angle: ", min(trainer_hip_angle_state2), max(trainer_hip_angle_state2))
print("Knee Angle: ", min(trainer_knee_angle_state2), max(trainer_knee_angle_state2))

print("\nState 3:-\n")
print("Elbow Angle: ", min(trainer_elbow_angle_state3), max(trainer_elbow_angle_state3))
print("Shoulder Angle: ", min(trainer_shoulder_angle_state3), max(trainer_shoulder_angle_state3))
print("Hip Angle: ", min(trainer_hip_angle_state3), max(trainer_hip_angle_state3))
print("Knee Angle: ", min(trainer_knee_angle_state3), max(trainer_knee_angle_state3))